In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ML_utils import load_data,load_data_test_set, total_day_eve_night_grouping
df =pd.read_csv('train.csv')

### UNCOMMENT THIS PART TO GROUP FEATURES
feature_engineering = True
df=total_day_eve_night_grouping(df,grouping=feature_engineering)
# exclude_list = ['state','area_code']

exclude_list = [feature_name for feature_name in df.columns if feature_name not in['number_vmail_messages','number_customer_service_calls','total_charges','churn']]

one_hot_on = True
normalize_on = True
oversample_on=True


###################
#### LOAD DATA ####

X_train,y_train,X_val,y_val,X_test,y_test,columns = load_data(df,exclude=exclude_list,one_hot=one_hot_on,normalize_=normalize_on,oversample=oversample_on)


X_train.shape



(2975, 3)

In [17]:
from ML_forest_utils import grid_search_random_forest,predict_forest



In [18]:
tree_in_forest_list=[10,30,60,90,120]
tree_depth_list=[3,6,9,12,15,18,25,50]
min_sample_split_list=[2,4,6,8,10,12,16,20]
results=grid_search_random_forest(X_train,y_train,X_val,y_val,tree_in_forest_list,tree_depth_list,min_sample_split_list)
print('best_acc_val:',results['best_acc_val'],'best_tree_in_forest:',results['best_tree_in_forest'],'best_tree_depth:',results['best_tree_depth'],'best_min_sample_split:',results['best_min_sample_split'])
print(columns)

tree_in_forest: 10 tree_depth: 3 min_sample_split: 2 accuracy: 0.9089481946624803
tree_in_forest: 10 tree_depth: 3 min_sample_split: 4 accuracy: 0.3877551020408163
tree_in_forest: 10 tree_depth: 3 min_sample_split: 6 accuracy: 0.3877551020408163
tree_in_forest: 10 tree_depth: 3 min_sample_split: 8 accuracy: 0.8492935635792779
tree_in_forest: 10 tree_depth: 3 min_sample_split: 10 accuracy: 0.6138147566718996
tree_in_forest: 10 tree_depth: 3 min_sample_split: 12 accuracy: 0.3893249607535322
tree_in_forest: 10 tree_depth: 3 min_sample_split: 16 accuracy: 0.8649921507064364
tree_in_forest: 10 tree_depth: 3 min_sample_split: 20 accuracy: 0.9058084772370487
tree_in_forest: 10 tree_depth: 6 min_sample_split: 2 accuracy: 0.9246467817896389
tree_in_forest: 10 tree_depth: 6 min_sample_split: 4 accuracy: 0.8854003139717426
tree_in_forest: 10 tree_depth: 6 min_sample_split: 6 accuracy: 0.8712715855572999
tree_in_forest: 10 tree_depth: 6 min_sample_split: 8 accuracy: 0.8492935635792779
tree_in_fore

In [19]:
### USE THE MODEL ON THE TEST SET
df_test=pd.read_csv('test.csv')

df_test=total_day_eve_night_grouping(df_test,grouping=feature_engineering)


X_test_output=load_data_test_set(df_test,exclude=exclude_list+['id'],one_hot=one_hot_on,normalize_=normalize_on)

y_pred_output=np.round(predict_forest(X_test_output,results['best_forest'],results['best_feature_indices']))

y_pred_output=np.where(y_pred_output>0.5, 'yes', 'no')
y_pred_output=y_pred_output.reshape(-1,)
id_column = np.arange(1, y_pred_output.shape[0] + 1)


# Create a DataFrame
df_output = pd.DataFrame({
    'id': id_column,
    'churn': y_pred_output
})


# Save the DataFrame as a CSV file
df_output.to_csv('output_random_forest.csv', index=False)